In [1]:
import pandas as pd
import requests

characters = pd.read_csv('vertex.csv')
characters = list(characters['Label'])

In [0]:
S = requests.Session()

def get_character_page(title):


  URL = "https://gameofthrones.fandom.com/api.php"

  PARAMS = {
      "action": "parse",
      "format": "json",
      "page": title,
      "prop": "wikitext",
      "formatversion": "2"
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()

  return DATA

character_contents = {}
for character in characters:
  character_contents[character] = get_character_page(character)

In [0]:
contents = {}
for title in character_contents:
  contents[title] = character_contents[title]['parse']['wikitext']['*']

In [0]:
contents_df = pd.DataFrame(contents.items(), columns=['wiki_title', 'wiki_content'])

In [44]:
contents_df

,wiki_title,wiki_content
0,Cassana Estermont,#REDIRECT[[Cassana Baratheon]]
1,Merchant captain,{{Character\n| Title = Merchant captain\n| Ima...
2,Fennesz,{{Character\n| Title=Fennesz\n| Image=Fennesz....
3,Musician 5 (The Door),{{DISPLAYTITLE:Musician 5 <small>(The Door)</s...
4,Valryon,{{Animal\n|Title = Valryon\n|Season = [[Season...
...,...,...
2041,Mordane,{{Character\n| Title=Mordane\n| Image=MordaneC...
2042,Great Master 5,{{Heraldry\n|image=SlaveMasters-Shield.PNG\n|l...
2043,Red Priestess (High Sparrow),{{DISPLAYTITLE:Red Priestess <small>(High Spar...
2044,Dareon,{{Heraldry\n|image=Night's-Watch-Main-Shield.P...


## Delete redirect links

In [61]:
not_redirected = contents_df[~contents_df.wiki_content.str.lower().str.contains('#redirect')]
not_redirected

,wiki_title,wiki_content
1,Merchant captain,{{Character\n| Title = Merchant captain\n| Ima...
2,Fennesz,{{Character\n| Title=Fennesz\n| Image=Fennesz....
3,Musician 5 (The Door),{{DISPLAYTITLE:Musician 5 <small>(The Door)</s...
4,Valryon,{{Animal\n|Title = Valryon\n|Season = [[Season...
5,Jaremy Mallister,{{Heraldry\n|image = House-Mallister-Main-Shie...
...,...,...
2041,Mordane,{{Character\n| Title=Mordane\n| Image=MordaneC...
2042,Great Master 5,{{Heraldry\n|image=SlaveMasters-Shield.PNG\n|l...
2043,Red Priestess (High Sparrow),{{DISPLAYTITLE:Red Priestess <small>(High Spar...
2044,Dareon,{{Heraldry\n|image=Night's-Watch-Main-Shield.P...


In [0]:
not_redirected.to_csv('not_redirected_titles.csv')


In [34]:
not_redirected = pd.read_csv('not_redirected_titles.csv')

## Filter pages without "{{Character" substring

In [35]:
only_characters = not_redirected[not_redirected.wiki_content.str.contains('{{Character')]

In [30]:
only_characters.iloc[:, [0,1]].to_csv('clean_vertex.csv', header=False, index=False)


In [31]:
edge = pd.read_csv('edge.csv')

In [32]:
edge = edge[edge.Target.isin(only_characters.iloc[:, 0].values) & edge.Source.isin(only_characters.iloc[:, 0].values)]

In [33]:
edge.to_csv('clean_edge.csv', index=False)